In [1]:
import matplotlib.pyplot as plt
import lesion
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.image as mpimg
import numpy as np
device = torch.device("cuda:0")
import unet
import utils as util
import torch.nn.functional as F
import residual_unet
from sklearn.metrics import jaccard_score as jsc
import ECE

In [2]:
class ToTensor_segmap(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, segmap):
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        # In this case, Channels is 1, so there is no need to swap since data is in HxW      
  
        segmap = np.array(segmap)
        return torch.from_numpy(segmap) / 255

#mean of dataset [0.6193, 0.6193, 0.6193]
#img net mean, [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]
image_transform = transforms.Compose([
        transforms.RandomGrayscale(1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ])
seg_transform = ToTensor_segmap()

In [3]:
batch_size = 8
dataset = lesion.LesionDataset("data",folder_name = 'test',joint_transform=False,img_transform=image_transform, seg_transform=seg_transform,verbose = True)
loader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=False, num_workers=4)
total = len(dataset)
n = len(loader)

In [5]:
file_name = "res_unet_all.pth"
model = torch.load("models/" + file_name)
model.eval()

DataParallel(
  (module): res_unet(
    (MaxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (drop): Dropout2d(p=0.5, inplace=False)
    (preprocess): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (down1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
     

In [6]:
preds = np.zeros((total, 2, 512, 1024))
targets = np.zeros((total, 512, 1024))

for i,data in enumerate(loader):
    image = data[0].to(device)
    segmap = data[1].numpy()
    output = model(image)
    preds[batch_size*i:batch_size*i + batch_size] = output.detach().cpu().numpy()
    targets[batch_size*i:batch_size*i + batch_size] = segmap



In [7]:
out = preds.argmax(axis=1)
def get_score(pred,t):
    jaccard = 0
    for i in range(total):
        jaccard += jsc(pred[i].reshape(-1),t[i].reshape(-1))
    jaccard /= total
    
    return jaccard,np.count_nonzero(pred==t)/pred.reshape(-1).shape[0]
        

In [ ]:
jaccard,accuracy =get_score(out,targets)
print("Jaccard Score:{},  Accuracy:{} ".format(jaccard,accuracy))

In [ ]:
jsc(out[3].reshape(-1),targets[3].reshape(-1))

In [ ]:
avg_ece = 0
criterion = ECE.ECELoss()
for i in range(600):
    m_i = torch.tensor(preds[i].reshape(-1,2)).to(device)
    t_i = torch.tensor(targets[i].reshape(-1)).to(device)
    avg_ece += criterion(m_i,t_i)

avg_ece/=600
avg_ece

In [ ]:

i=90
plt.title("Ground Truth")
plt.imshow(targets[i])


In [ ]:
plt.title("Res U-net Concat")
plt.imshow(out[i])

In [ ]:
seg_show(targets[14])